In [ ]:
pip install fasttext

In [19]:
import fasttext
from huggingface_hub import hf_hub_download
import pandas as pd
from tqdm import tqdm
import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import csv
import sys

In [ ]:
model_path = hf_hub_download(repo_id="facebook/fasttext-ru-vectors", filename="model.bin") # Загрузка модели
model = fasttext.load_model(model_path)

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
def preprocess(text):
    """
    Обработка данных
    """
    text = list(filter(str.isalpha, word_tokenize(text.lower()))) # Токенизация
    text = list(lemmatizer.lemmatize(word) for word in text) # Лемматизация
    text = list(word for word in text if word not in stop_words) # Удаление стоп-слов
    return ' '.join(text)
stop_words = set(stopwords.words('english') + stopwords.words('russian') + ['xn'])
lemmatizer = WordNetLemmatizer()

In [35]:
df = pd.DataFrame() # Запись обработанного текста
lst = [] # Запись эмбеддингов

In [ ]:
csv.field_size_limit(3000000000000000000) # Технические параметры

In [ ]:
with open('/content/Test/parsing.csv', newline='') as f:
  reader = csv.reader(f)
  for row in tqdm(reader):
    clear_row = re.sub(r'\s+', ' ', ' '.join(row).replace('\n', ''))[2:]
    if clear_row == 'ERROR ERROR ERROR ERROR ERROR ERROR' or len(clear_row) == 0: # Отсечение строк, о которых не получить информацию
      continue
    row = preprocess(clear_row) # Обработка данных
    df = pd.concat((df, pd.DataFrame({'text': [row]}))) # Добавление текстовой информации ссылки в df
    lst.append(model[row]) # Добавление эмбеддинга текстовой информации в lst

In [ ]:
df.to_csv('text.csv') # Сохранение текста
np.save('embed.npy', lst) # Сохранение эмбеддинга